### **Objectives**

In this notebook I will analyze the file 'business.json'.

In it we have business data, including location data, attributes, categories and more.

I will perform the manipulation and treatment of the data, in addition to generating new features, so that they can be used as inputs of the future model.

#### 1. Importing the libraries

In [1]:
import os
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder

#### 2. Reading the "business.json" file and creating the dataset

In [3]:
# Reading the "business.json" file and creating the dataset
business = pd.read_json('..\\INPUT\\business.json', lines=True)

# Displaying the first few lines of the dataset to confirm that the file was read correctly
business.head(3)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


#### 3. Data Processing + Feature Engineering

In [4]:
# Checking dataset details (missing values, data types, and number of records)

business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


* Each row of this dataset contains information regarding a SINGLE BUSINESS and that we have 150.346 registered businesses.

* Note also the presence of missing values in the last 3 features of the dataset (attributes, categories and hours).

For this project I won't use the features below:

- 'business_id': They are the unique identifiers of each business
- 'name' e 'address': I'll focus on more relevant features like 'city' and 'state'
- 'postal_code', 'latitude' e 'longitude': Related to the 'address' feature

In [5]:
# Dropping the unnecessary features

features = ['business_id','name','address','postal_code','latitude','longitude']
business.drop(features, axis=1, inplace=True)

Some features may have their types changed to take up less memory space.

After dropping the unnecessary features the dataset was taking up 9.6 MB of memory. Let's see how it will look after the transformations.

In [6]:
# Converting features to optimize memory usage

# Converting from 'string' to 'category'
business[['city','state']] = business[['city','state']].astype('category')

# Converting from 'float64' to 'float16' because the values are no more than 5
business['stars'] = business['stars'].astype('float16')

# Converting from 'int64' to 'int8' because the values are only 0 and 1
business['is_open'] = business['is_open'].astype('int8')

# Converting from 'int64' to 'int16'
business['review_count'] = business['review_count'].astype('int16')

In [7]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   city          150346 non-null  category
 1   state         150346 non-null  category
 2   stars         150346 non-null  float16 
 3   review_count  150346 non-null  int16   
 4   is_open       150346 non-null  int8    
 5   attributes    136602 non-null  object  
 6   categories    150243 non-null  object  
 7   hours         127123 non-null  object  
dtypes: category(2), float16(1), int16(1), int8(1), object(3)
memory usage: 4.6+ MB


> We observed a 50% reduction in memory usage by the dataset.

The remaining three features cannot be converted as they contain dictionaries in their values and will be dealt with individually below.

I decided to start by treating the **'categories'** feature.

In [8]:
# Displaying a sample of the contents of the "categories" feature

pd.set_option('display.max_colwidth', None)
pd.DataFrame(business['categories'].head(10))

,categories
0,"Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists"
1,"Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services"
2,"Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores"
3,"Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries"
4,"Brewpubs, Breweries, Food"
5,"Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants"
6,"Sporting Goods, Fashion, Shoe Stores, Shopping, Sports Wear, Accessories"
7,"Synagogues, Religious Organizations"
8,"Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek"
9,"Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food"


In [9]:
# Checking the total of SINGLE categories registered

print("Total of single categories:",business["categories"].nunique())

Total of single categories: 83160


Each row contains several categories, separated by comma and in string format, associated with a **single business**.

For this reason we observe such a high amount of unique categories present in this feature (83.160).

In [10]:
# Checking the top 10 categories that have the most registered businesses

business['categories'].value_counts().head(10)

Beauty & Spas, Nail Salons    1012
Restaurants, Pizza             935
Nail Salons, Beauty & Spas     934
Pizza, Restaurants             823
Restaurants, Mexican           728
Restaurants, Chinese           708
Mexican, Restaurants           672
Chinese, Restaurants           651
Food, Coffee & Tea             508
Beauty & Spas, Hair Salons     493
Name: categories, dtype: int64

We can notice that there are inconsistencies such as 'Beauty & Spas, Nail Salons' and 'Nail Salons, Beauty & Spas' being identified as unique categories just by the fact that they are ordered differently.

I'll split the strings to find out how many really unique categories we have in the dataset. I'll also put them in alphabetical order to solve the problem identified above.

In [11]:
# Extracting the unique categories, performing the sorting and inserting the new feature into the dataframe

# Classifying the null categories as 'Unknown'
business["categories"].fillna(value='Unknown', inplace=True)

unique_categories = [] #List that will hold the unique categories

def category_processing(feature):

    """
    Performs the identification of the unique values and also performs the sorting

    :param feature: pandas Series
    :return pandas Dataframe
    """

    processed_categories = [] #List that will hold each of the categories after treatment

    # Scrolling through each value of the 'categories' feature
    for category in feature:
        # Handling strings (removing spaces and sorting)
        new_category = category.replace(' ','').split(',')
        new_category.sort()
        # Checking that the value is not in the 'unique_categories' list
        # and appending to the list if the condition is met
        for item in new_category:
            if item not in unique_categories:
                unique_categories.append(item)
        # Performing further processing after sorting
        # and appending to the list 'processed_categories'
        new_category = str(new_category).replace('[','').replace(']','').replace("'",'')
        processed_categories.append(new_category)
    
    return pd.DataFrame(processed_categories)

# Running the processing function and updating the 'categories' feature with the processed values
business['categories'] = category_processing(business['categories'])

In [12]:
# Checking the total of REALLY SINGLE categories registered after the initial treatment

print('Total of unique categories found:', len(unique_categories))

Total of unique categories found: 1312


In [13]:
# Displaying the total of categories found in the "categories" feature (after sorting the values)

print("Total categories found in the 'categories' feature: ",business['categories'].nunique())

Total categories found in the 'categories' feature:  44741


Even after performing the alphabetical sorting we noticed a high number of categories (44.741) because they can have different quantities for the same business.

Eg: "Acupunture, Doctors" will differ from "Acupunture, Doctors, Health&Medical".

So I decided to perform a kind of OneHotEncoding manually, using ONLY THE SINGLE CATEGORIES found previously (1.312 categories).

In [14]:
# Re-processing the categories to create new features in OneHotEncoding format

def one_hot_categories(feature):

    """
    Create new features for the 'categories' in the OneHotEncoding format

    :param feature: pandas Series
    :return pandas Dataframe
    """
    
    ohe_categories = []
    
    for category in feature:
        category = category.split(',')
        row_dict = {}
        for item in category:
            item = item.replace(' ','')
            item = item.replace("'",'')
            row_dict['category_' + item] = 1
        ohe_categories.append(row_dict)
    
    return pd.DataFrame(ohe_categories)

# Running the processing function
df_categories = one_hot_categories(business['categories'])

# Filling the null values with zeros
df_categories.fillna(0, inplace=True)

As a result I got 1.312 more features (one for each **really unique** category).

In [15]:
# Displaying a sample of the content after treatment

df_categories.head(5)

,category_Acupuncture,category_Doctors,category_Health&Medical,category_Naturopathic/Holistic,category_Nutritionists,category_TraditionalChineseMedicine,category_LocalServices,category_MailboxCenters,category_Notaries,category_PrintingServices,...,category_AviationServices,category_ConveyorBeltSushi,category_DialysisClinics,category_Makerspaces,category_NaturalGasSuppliers,category_Hospitalists,category_SerboCroatian,category_CeremonialClothing,category_BubbleSoccer,category_TradeFairs
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


To **optimize** the result of the above process and **take up less memory space** I decided to convert all the generated feature values to 'int8' format (they were in 'float64' format) because they only contain the values 0 and 1.

In [16]:
# Creating a function to display the total amount of memory used by a particular Dataframe

def mem_usage(df):

    """
    Calculates the total amount of memory used by a Dataframe

    :param df: pandas Dataframe
    :return float
    """

    amount_used = 0

    for i in df:
        amount_used += df[i].memory_usage(index=False)

    return amount_used * (10**-6) # Return the value in Megabytes

In [17]:
print(f'Amount of memory used before optimizing: {mem_usage(df_categories)} Megabytes')

# Converting dataset values from 'float64' to 'int8
df_categories = df_categories.astype('int8')

print(f'Amount of memory used after optimizing: {mem_usage(df_categories)} Megabytes')

Amount of memory used before optimizing: 1578.031616 Megabytes
Amount of memory used after optimizing: 197.253952 Megabytes


> Notice the drastic reduction in the amount of memory used!!!

In [18]:
# Concatenating new features to the dataframe

business = pd.concat([business, df_categories], axis=1)

Next I will perform the handling of the **'attributes'** feature.

In [19]:
# Displaying a sample content of the "attributes" feature

pd.DataFrame(business['attributes'].head(10))

,attributes
0,{'ByAppointmentOnly': 'True'}
1,{'BusinessAcceptsCreditCards': 'True'}
2,"{'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'CoatCheck': 'False', 'RestaurantsTakeOut': 'False', 'RestaurantsDelivery': 'False', 'Caters': 'False', 'WiFi': 'u'no'', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'WheelchairAccessible': 'True', 'HappyHour': 'False', 'OutdoorSeating': 'False', 'HasTV': 'False', 'RestaurantsReservations': 'False', 'DogsAllowed': 'False', 'ByAppointmentOnly': 'False'}"
3,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}"
4,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}"
5,"{'BusinessParking': 'None', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsAttire': 'u'casual'', 'OutdoorSeating': 'True', 'RestaurantsReservations': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'Alcohol': 'u'none'', 'Ambience': 'None', 'GoodForKids': 'True', 'RestaurantsPriceRange2': '1', 'ByAppointmentOnly': 'False', 'CoatCheck': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'True', 'WiFi': 'u'no'', 'WheelchairAccessible': 'True', 'HasTV': 'True', 'HappyHour': 'False', 'DriveThru': 'True', 'BikeParking': 'False'}"
6,"{'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'BikeParking': 'True', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}'}"
7,None
8,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', 'RestaurantsAttire': 'u'casual'', 'RestaurantsDelivery': 'False', 'RestaurantsTakeOut': 'True', 'HasTV': 'True', 'NoiseLevel': 'u'average'', 'BusinessAcceptsCreditCards': 'True', 'OutdoorSeating': 'True', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False}', 'RestaurantsPriceRange2': '1', 'GoodForKids': 'True', 'WiFi': 'u'free'', 'RestaurantsReservations': 'False', 'RestaurantsGoodForGroups': 'True'}"
9,"{'RestaurantsAttire': ''casual'', 'RestaurantsGoodForGroups': 'False', 'BusinessAcceptsCreditCards': 'True', 'OutdoorSeating': 'True', 'GoodForKids': 'True', 'Alcohol': 'u'none'', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'ByAppointmentOnly': 'False', 'WiFi': 'u'no'', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'HasTV': 'True', 'RestaurantsDelivery': 'True', 'CoatCheck': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'DriveThru': 'True', 'HappyHour': 'False', 'WheelchairAccessible': 'True'}"


Note that each business has unique attributes and these are inserted in dictionaries that, in some cases, have other dictionaries as values.

E.g: 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}'

This will take a bit more work to extract the unique attributes and create new features, but let's go for it!

In [20]:
# Processing the attributes to create new features in OneHotEncoding format

# Classifying the null attributes as 'Unknown'
business["attributes"].fillna(value='Unknown', inplace=True)

def one_hot_attributes(feature):
    
    """
    Create new features for the 'attributes' in the OneHotEncoding format

    :param feature: pandas Series
    :return pandas Dataframe
    """

    ohe_attributes = []
    subattribute_values = []

    for attribute in feature:
        row_dict = {}
        att_dict = {}
        if type(attribute) is dict:
            for key, value in attribute.items():
                if '{' in value:
                    value = ast.literal_eval(value)
                    for subkey, subvalue in value.items():
                        att_dict['subattribute_' + key + '_' + subkey] = subvalue
                    row_dict['attribute_' + key] = 1
                else:
                    row_dict['attribute_' + key] = value
        else:
            row_dict['attribute_' + attribute] = 1

        ohe_attributes.append(row_dict)
        subattribute_values.append(att_dict)
            
    attributes_df = pd.DataFrame(ohe_attributes)
    subattributes_df = pd.DataFrame(subattribute_values)

    return pd.concat([attributes_df,subattributes_df], axis=1)

# Running the processing function
df_attributes = one_hot_attributes(business['attributes'])

# Filling the null values with zeros
df_attributes.fillna(0, inplace=True)

With the function above I was able to split the attributes and create something like a OneHotEncoding, but notice that there are values like "True" and "False" that could be displayed as 1 and 0.

There are also attributes with distinct values (e.g. attribute_RestaurantsPriceRange2 and attribute_WiFi).

In the following I will deal with these factors.

In [21]:
# Displaying a sample result of the first treatment of the "attributes" feature

df_attributes.head(5)

,attribute_ByAppointmentOnly,attribute_BusinessAcceptsCreditCards,attribute_BikeParking,attribute_RestaurantsPriceRange2,attribute_CoatCheck,attribute_RestaurantsTakeOut,attribute_RestaurantsDelivery,attribute_Caters,attribute_WiFi,attribute_BusinessParking,...,subattribute_HairSpecializesIn_kids,subattribute_HairSpecializesIn_perms,subattribute_HairSpecializesIn_asian,subattribute_DietaryRestrictions_dairy-free,subattribute_DietaryRestrictions_gluten-free,subattribute_DietaryRestrictions_vegan,subattribute_DietaryRestrictions_kosher,subattribute_DietaryRestrictions_halal,subattribute_DietaryRestrictions_soy-free,subattribute_DietaryRestrictions_vegetarian
0,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,False,True,True,2,False,False,False,False,u'no',1,...,0,0,0,0,0,0,0,0,0,0
3,False,False,True,1,0,True,False,True,u'free',1,...,0,0,0,0,0,0,0,0,0,0
4,0,True,True,0,0,True,0,False,0,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Re-processing the attributes

def extra_attributes_processing(feature):

    """
    Reworks the 'attributes' feature

    :param feature: pandas Series
    :return value
    """
    
    if isinstance(feature, str):
        if feature == 'None' or feature == 'False':
            return 0
        elif feature == 'True':
            return 1
        else:
            return feature.replace("'",'')
        
    elif isinstance(feature, bool):
        if feature == True:
            return 1
        else:
            return 0
        
    elif np.isnan(feature):
        return 0
    
    else:
        return feature
    
df_attributes2 = pd.DataFrame()

# Running the processing function
for feature in df_attributes:
    df_attributes2[feature] = pd.DataFrame(df_attributes[feature].apply(extra_attributes_processing))

In [23]:
# Displaying a sample result of the second treatment of the "attributes" feature

df_attributes2.head(5)

,attribute_ByAppointmentOnly,attribute_BusinessAcceptsCreditCards,attribute_BikeParking,attribute_RestaurantsPriceRange2,attribute_CoatCheck,attribute_RestaurantsTakeOut,attribute_RestaurantsDelivery,attribute_Caters,attribute_WiFi,attribute_BusinessParking,...,subattribute_HairSpecializesIn_kids,subattribute_HairSpecializesIn_perms,subattribute_HairSpecializesIn_asian,subattribute_DietaryRestrictions_dairy-free,subattribute_DietaryRestrictions_gluten-free,subattribute_DietaryRestrictions_vegan,subattribute_DietaryRestrictions_kosher,subattribute_DietaryRestrictions_halal,subattribute_DietaryRestrictions_soy-free,subattribute_DietaryRestrictions_vegetarian
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,2,0,0,0,0,uno,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,1,0,1,ufree,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


The processing is almost complete, but we still have features like "attribute_WiFi" that have values in string format (E.g: 'uno', 'ufree').

I used the code below to identify these features so that I can perform specific processing for each case as needed.

In [24]:
# Identifying features that need special / additional treatment

special_features = []
for feature in df_attributes2:
    for val in df_attributes2[feature].values:
        if val not in [0,1]:
            if feature not in special_features:
                special_features.append(feature)

special_features

['attribute_RestaurantsPriceRange2',
 'attribute_WiFi',
 'attribute_Alcohol',
 'attribute_RestaurantsAttire',
 'attribute_NoiseLevel',
 'attribute_Smoking',
 'attribute_BYOBCorkage',
 'attribute_AgesAllowed']

In [26]:
# Analyzing the content of each feature to decide how the treatments will be

for feature in special_features:
    print(f'{feature} has the values {df_attributes2[feature].unique()}')

attribute_RestaurantsPriceRange2 has the values [0 '2' '1' '3' '4']
attribute_WiFi has the values [0 'uno' 'ufree' 'free' 'no' 'upaid' 'paid']
attribute_Alcohol has the values [0 'unone' 'ufull_bar' 'none' 'full_bar' 'ubeer_and_wine' 'beer_and_wine']
attribute_RestaurantsAttire has the values [0 'ucasual' 'casual' 'uformal' 'dressy' 'udressy' 'formal']
attribute_NoiseLevel has the values [0 'uaverage' 'uquiet' 'average' 'uloud' 'uvery_loud' 'quiet' 'very_loud'
 'loud']
attribute_Smoking has the values [0 'uno' 'uoutdoor' 'uyes' 'outdoor' 'no']
attribute_BYOBCorkage has the values [0 'yes_free' 'no' 'yes_corkage' 'uyes_free' 'uyes_corkage' 'uno']
attribute_AgesAllowed has the values [0 'u21plus' 'uallages' 'u18plus']


I decided to use Pandas "cat.codes" function to turn each category into a number simply and quickly.

In [27]:
for feature in special_features:
    df_attributes2[feature] = df_attributes2[feature].astype('category')
    df_attributes2[feature] = df_attributes2[feature].cat.codes

Again I will optimize the memory usage because the values in this Dataframe are between 0 and 9 and there is no need to store them as 'float64'.

In [28]:
print(f'Amount of memory used before optimizing: {mem_usage(df_attributes2)} Megabytes')

# # Converting dataset values from 'float64' to 'int8'
df_attributes2 = df_attributes2.astype('int8')

print(f'Amount of memory used after optimizing: {mem_usage(df_attributes2)} Megabytes')

Amount of memory used before optimizing: 98.626976 Megabytes
Amount of memory used after optimizing: 13.380794 Megabytes


In [29]:
# Concatenating new features to the dataframe

business = pd.concat([business, df_attributes2], axis=1)

In the next step I will perform the treatment of the **'hours'** feature.

In [30]:
# Displaying a sample of the contents of the "hours" feature

business['hours'].head(10)

0                                                                                                                                                                       None
1                            {'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}
2         {'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday': '8:0-22:0', 'Friday': '8:0-23:0', 'Saturday': '8:0-23:0', 'Sunday': '8:0-22:0'}
3         {'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}
4                                                 {'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}
5           {'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturda

Notice that for each day of the week we have the opening and closing time as a single string. We also have null values in this feature.

So I decided to use similar processing as before to create new features.

In [31]:
# Processing the 'hours' to create new features in OneHotEncoding format

def one_hot_hours(feature):
    
    """
    Create new features for the 'hours' in the OneHotEncoding format

    :param feature: pandas Series
    :return pandas Dataframe
    """

    ohe_hours = []

    for item in feature:
        row_dict = {}
        if item == None :
            row_dict['hours_None'] = 1
        else:
            for key, value in item.items():
                row_dict['hours_' + key] = value

        ohe_hours.append(row_dict)

    hours_df = pd.DataFrame(ohe_hours)
    return hours_df

# Running the processing function
df_hours = one_hot_hours(business['hours'])

# Filling the null values with zeros
df_hours.fillna(0, inplace=True)

As you can see below, we have created features for each day of the week and inserted the respective times into each of them. But this way it will still be difficult to use this information in a future model.

In [32]:
# Displaying a sample of the result

df_hours.head(5)

,hours_None,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday
0,1.0,0,0,0,0,0,0,0
1,0.0,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,0
2,0.0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,0.0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,0.0,0,0,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0


Again I'll use Pandas "cat.codes" function to turn each item into a number simply and quickly.

In [33]:
for feature in df_hours:
    df_hours[feature] = df_hours[feature].astype('category')
    df_hours[feature] = df_hours[feature].cat.codes

In [34]:
# Displaying a sample of the result

df_hours.head(5)

,hours_None,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday
0,1,0,0,0,0,0,0,0
1,0,1,1192,1177,1234,1296,1171,0
2,0,1115,1205,1191,1249,1315,1205,1111
3,0,996,1074,1061,1115,1180,1072,999
4,0,0,0,402,536,328,322,298


In [35]:
# Concatenating new features to the dataframe

business = pd.concat([business, df_hours], axis=1)

The last treatment I'll perform is the conversion of the 'city' and 'state' features using LabelEncoder.

In [36]:
features = ['city','state']
df_city_state = pd.DataFrame()

for feature in features:
    le = LabelEncoder()
    le.fit(business[feature])
    le_results = le.transform(business[feature])    

    df_city_state['le_' + feature] = le_results

In [37]:
# Optimizing the memory usage again

df_city_state['le_city'] = df_city_state['le_city'].astype('int16')
df_city_state['le_state'] = df_city_state['le_state'].astype('int8')

In [38]:
# Concatenating new features to the dataframe

business = pd.concat([business, df_city_state], axis=1)

In [39]:
# Dropping the features 'categories', 'attributes' and 'hours' as new features were generated for them

business.drop(['categories','attributes','hours'], axis=1, inplace=True)

I decided to export the final dataframe to a Pickle file because it took up half the disk space that a CSV file would and also kept the formats of the values (int8, category, etc) which will make it possible to read the file much faster in the next stages of the project.

In [90]:
# Exporting the final dataset to a pickle file

business.to_pickle('../INPUT/business_EDITED.pkl')

As a final result of this step I obtained a dataset with 1.416 features.

Throughout the project I will be able to apply dimensionality reduction techniques to the dataset if necessary.

> [Next step - EDA](./3_Business_EDA.ipynb)